In [1]:
import findspark
findspark.init('/opt/spark/spark-3.2.1-bin-hadoop3.2')

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
## Import SparkSession
from pyspark.sql import SparkSession

## Create SparkSession 
spark = SparkSession.builder \
      .master("local") \
      .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/company.myCollection") \
      .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/company.myCollection") \
      .appName("Spark-Kafka-Spark-MongoDB-Streamlit") \
      .getOrCreate() 

22/06/18 11:03:15 WARN Utils: Your hostname, ubuntu-machine resolves to a loopback address: 127.0.1.1; using 192.168.1.134 instead (on interface wlp0s20f3)
22/06/18 11:03:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/18 11:03:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.csv('hdfs://localhost:9000/data/data.csv', header=True, inferSchema=True)
df.printSchema()
df.show()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- job: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- country: string (nullable = true)
 |-- car_model_year: integer (nullable = true)
 |-- car_make: string (nullable = true)

+---+----------+--------------------+------+------+--------------------+--------------+-------------+
| id|first_name|                 job|gender|salary|             country|car_model_year|     car_make|
+---+----------+--------------------+------+------+--------------------+--------------+-------------+
|  1|     Lorri|Environmental Spe...|Female|7335.7|               China|          1991|   Mitsubishi|
|  2|      Leon|   Account Executive|  Male|8071.1|             Tunisia|          1984|         Ford|
|  3|      Eryn|  Environmental Tech|Female|1905.6|Democratic Republ...|          1997|         Audi|
|  4|    Nicoli|     General Manager|Female|3724.7|      Czech Repub

In [5]:
df = df.toPandas()

id_dict = df.to_dict(orient='id')

In [6]:
from kafka import KafkaProducer

topic = 'data'
bootstrap_servers = 'localhost:9092'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

In [ ]:
for i in range(len(id_dict)):
    future = producer.send(topic, bytes(str(id_dict[i]), 'utf-8'))
    print(f'Sending msg: {id_dict[i]}')
    result = future.get(timeout=60)